In [16]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import zipfile
import io
import xlsxwriter
import time

In [158]:
#Funções:

###Função requisicao dados CVN
# Função para fazer a requisição e tratar os possíveis erros
def fazer_requisicao(url):
    try:
        # Realizar a requisição
        response = requests.get(url)

        # Verificar o status code da resposta
        if response.status_code == 200:
            print('Requisição bem-sucedida para '+mes+'/'+ano+'.')
            return response
        elif response.status_code == 404:
            print('Página não encontrada (erro 404). Verifique a devida publicação na CVM disponível no site https://dados.cvm.gov.br/dados/FI/DOC/')
            return  response # Interrompe o código aqui se o status code for 404
        else:
            print(f'Erro inesperado: Status code {response.status_code}')
            return  # Interrompe o código em caso de status code diferente de 200 e 404

    except requests.ConnectionError:
        print('Erro de conexão: Verifique sua conexão de internet.')
    except requests.Timeout:
        print('Erro de timeout: A requisição excedeu o tempo limite.')
    except requests.RequestException as e:
        print(f'Erro na requisição: {e}')


def coleta(objetivo, mes, ano):
    
    """
    Esta função captura dados do site da CVM.

    Parâmetros:
    objetivo(string): qual o tipo de informação deseja 'lamina' ou 'informativo'.
    mes(int): qual o mês do dado.
    ano(int): qual o ano do dado.
    
    Retorno:

    df com o dado CVM 
    
    
    """
    objetivo = objetivo
    mes = mes
    ano = ano
    ano = str(ano)
    mes = str(mes).zfill(2) 
    informativo_dia  = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_'+ano+mes+'.zip'
    lamina =  'https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/lamina_fi_'+ano+mes+'.zip'
    arquivo_inf_diario = 'inf_diario_fi_'+ano+mes+'.csv'
    arquivo_lamina = 'lamina_fi_'+ano+mes+'.csv'
    

    if objetivo =='informativo':
        link_arq = informativo_dia
        tp_arq = arquivo_inf_diario
    elif objetivo =='lamina':
        tp_arq = arquivo_lamina
        link_arq = lamina

    print('{}\n{}'.format(link_arq, tp_arq))
    r = fazer_requisicao(link_arq)
    
    if r.status_code == 404:
        print()
        
    else:

        zf = zipfile.ZipFile(io.BytesIO(r.content))

        arquivo = tp_arq

        zf = zf.open(arquivo)

        lines = zf.readlines()
        lines =[i.strip().decode('ISO-8859-1') for i in lines]
        lines = [i.split(';') for i in lines]

        df = pd.DataFrame(lines[1:], columns = lines[0])
        return df


In [159]:
coleta('lamina',3,2024)

https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/lamina_fi_202403.zip
lamina_fi_202403.csv
Página não encontrada (erro 404). Verifique a devida publicação na CVM disponível no site https://dados.cvm.gov.br/dados/FI/DOC/

